### Protocols

Python is a protocol based language.

If you're coming from Java, you can think of protocols the same way you think of interfaces.

Except Python does not have this very strict idea of an interface.

You simply add some functions to your class using a specific name, and if Python finds it there, it will use it. The onus is on you to get the naming right, and correctly implementing any specific set of these functions that loosely make up the protocol yourself.

#### The `str` and `repr` Protocols

Let's take a look at a very simple example.

When we have an object, we can as for it's string representation in two ways:

In [2]:
a = 10

In [3]:
str(a)

'10'

In [4]:
repr(a)

'10'

These look identical, but that is not always the case. In general `str` is used for end-user display, and `repr` is used for development (or debugging) display.

For example:

In [6]:
from fractions import Fraction

In [7]:
f = Fraction(1, 2)

In [8]:
str(f)

'1/2'

In [9]:
repr(f)

'Fraction(1, 2)'

Each class may implement it's own mechanism for returning a value for either `str` or `repr`.

This is done by implementing the correct protocol.

Let's create our own class and implement both the `str` and the `repr` protocols:

In [10]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name.strip()
    
    def __repr__(self):
        return f"Person(name='{self.name}')"

As you can see we simply implemented to specially name instance methods: `__str__` and `__repr__`.

Let's use them:

In [11]:
p = Person('Isaac Newton')

Now these are just instance methods, and can be called that way:

In [14]:
p.__str__()

'Isaac Newton'

In [15]:
p.__repr__()

"Person(name='Isaac Newton')"

But, because of the special names we used, when we use the `str()` and `repr()` functions, Python will find and use our custom `__str__` and `__repr__` methods instead:

In [16]:
str(p)

'Isaac Newton'

In [17]:
repr(p)

"Person(name='Isaac Newton')"

In Python, every class directly or indirectly, inherits from the `object` class. This class provides standard implementations for a lot of protocols.

In [18]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [19]:
p = Point(1, 2)

In [20]:
str(p)

'<__main__.Point object at 0x7fc8e855ebb0>'

In [21]:
repr(p)

'<__main__.Point object at 0x7fc8e855ebb0>'

As you can see the default string and representations simply document the class that was used to create that object, and the memory address of the instance. As you saw, we can override this default behavior by implementing our own special functions.

#### The `addition` Protocol

When we write something like this in Python:

In [23]:
1 + 2

3

What is actually happening, is that integres implement the addition protocol, and when Python sees

```
1 + 2
```

it actually uses the addition protocol defined by integers to evaluate that statement.

We can implement this protocol in our custom classes too.

Let's start by creating a basic vector class:

In [25]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Vector({self.x}, {self.y})"
    
    def __str__(self):
        return f"({self.x}, {self.y})"

In [26]:
v1 = Vector(1, 2)
v2 = Vector(10, 20)

We implemented the str and repr protocols, so we can do this:

In [28]:
print(str(v1))
print(repr(v1))

(1, 2)
Vector(1, 2)


But we cannot add those two vectors:

In [29]:
v1 + v2

TypeError: unsupported operand type(s) for +: 'Vector' and 'Vector'

As you can see Python is telling us it does not know how to add two `Vector` instances together.

We can tell Python how to do that, by simply implementin the `add` protocol:

In [30]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Vector({self.x}, {self.y})"
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __add__(self, other):
        new_x = self.x + other.x
        new_y = self.y + other.y
        return Vector(new_x, new_y)

Note: technically it would be better to check that `other` is also a `Vector` instance, but let's ignore that for now.

In [32]:
v1 = Vector(1, 2)
v2 = Vector(10, 20)

And now we can add those two vectors together:

In [34]:
v1 + v2

Vector(11, 22)

Ok, let's just go back and fix the `__add__` method, to at least make sure we are adding two vectors, because here's what happens right now:

In [35]:
v1 + 10

AttributeError: 'int' object has no attribute 'x'

In fact, the weird things is that if we have another object with those `x` and `y` attributes, the addition may actually work!

In [41]:
class NotAVector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.x = z

In [42]:
nav = NotAVector(10, 20, 30)

In [43]:
v1 + nav

Vector(31, 22)

So, we may want to restrict our addition to only two vectors:

In [48]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Vector({self.x}, {self.y})"
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __add__(self, other):
        if not isinstance(other, Vector):
            raise TypeError('Addition is only supported between two Vector instances.')
        new_x = self.x + other.x
        new_y = self.y + other.y
        return Vector(new_x, new_y)

In [49]:
v1 = Vector(1, 2)
v2 = Vector(10, 20)
nav = NotAVector(10, 20, 30)

In [50]:
v1 + v2

Vector(11, 22)

In [51]:
v1 + nav

TypeError: Addition is only supported between two Vector instances.

In [52]:
v1 + 10

TypeError: Addition is only supported between two Vector instances.

but what if we wanted to support something like this:

In [53]:
v1 + (10, 20)

TypeError: Addition is only supported between two Vector instances.

or

In [54]:
v1 + [10, 20]

TypeError: Addition is only supported between two Vector instances.

We can enhance our `__add__` method to allow this:

In [55]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Vector({self.x}, {self.y})"
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __add__(self, other):
        if isinstance(other, (list, tuple)) and len(other) >= 2:
            new_x = self.x + other[0]
            new_y = self.y + other[1]
        elif isinstance(other, Vector):
            new_x = self.x + other.x
            new_y = self.y + other.y
        else:
            raise TypeError(f"Unsupported type for Vector addition: {type(other)}")
        return Vector(new_x, new_y)

In [56]:
v1 = Vector(1, 2)
v2 = Vector(10, 20)
nav = NotAVector(10, 20, 30)

In [57]:
v1 + v2

Vector(11, 22)

In [58]:
v1 + (100, 200)

Vector(101, 202)

In [59]:
v1 + [100, 200]

Vector(101, 202)

In [60]:
v1 + nav

TypeError: Unsupported type for Vector addition: <class '__main__.NotAVector'>

#### Other Protocols

Most of the operators in Python, as well as various behavior traits of objects, are controlled in custom classes using these protocols, which you can find documented here:

https://docs.python.org/3/reference/datamodel.html